In [1]:
import sys
sys.path.append('../scripts/')
from utility import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10 GiB')
    .config('spark.shuffle.file.buffer', '64k')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/14 10:54:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 1. Import Data

In [4]:
print("================ Mobility ================")
rrm = read_file(spark, 'Retail and Recreation.csv', type='csv')

print("================ Cases ================")
ncd = read_file(spark, 'Net cases  day.csv', type='csv')

================ Mobility ================
|> Loading File...
|> Loading Finished!
-RECORD 0--------
 Date | 21/02/20 
 AUS  | 1        
 NSW  | 1        
 VIC  | 1        
 QLD  | -1       
 SA   | 2        
 WA   | 2        
 TAS  | -1       
 NT   | 4        
 ACT  | 4        
only showing top 1 row

================ Cases ================
|> Loading File...
|> Loading Finished!
-RECORD 0-------------------
 Date            | 23/10/20 
 Net cases / day | 18       
only showing top 1 row



### 2. 处理日期

In [5]:
from pyspark.sql import functions as F
# from pyspark.sql.functions import to_date, to_timestamp
rrm_new = rrm.withColumn(
    'Date',
    F.to_date('Date', 'dd/MM/yy')
)
rrm_new.show(5)

+----------+---+---+---+---+---+---+---+---+---+
|      Date|AUS|NSW|VIC|QLD| SA| WA|TAS| NT|ACT|
+----------+---+---+---+---+---+---+---+---+---+
|2020-02-21|  1|  1|  1| -1|  2|  2| -1|  4|  4|
|2020-02-22|  1|  1|  1| -1|  2|  1| -1|  5|  6|
|2020-02-23|  1|  0|  1| -1|  2|  2| -1|  6|  7|
|2020-02-24|  1|  1|  1| -1|  2|  2| -1|  6|  7|
|2020-02-25|  1|  1|  2| -1|  3|  1| -1|  7|  8|
+----------+---+---+---+---+---+---+---+---+---+
only showing top 5 rows



In [6]:
ncd_new = ncd.withColumn(
    'Date',
    F.to_date('Date', 'dd/MM/yy')
)
ncd_new.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Net cases / day: string (nullable = true)



### 3. When we looking for the null value of this external data-set, we find one data with null `Date`, so fill this specific table. 

In [7]:
import pandas as pd
rrm_new_df = rrm_new.toDF('Date', 'AUS', 'NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT')
rrm_new_df = pd.DataFrame(rrm_new_df.collect(), \
    columns=['Date', 'AUS', 'NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT'])
rrm_new_df

,Date,AUS,NSW,VIC,QLD,SA,WA,TAS,NT,ACT
0,2020-02-21,1,1,1,-1,2,2,-1,4,4
1,2020-02-22,1,1,1,-1,2,1,-1,5,6
2,2020-02-23,1,0,1,-1,2,2,-1,6,7
3,2020-02-24,1,1,1,-1,2,2,-1,6,7
4,2020-02-25,1,1,2,-1,3,1,-1,7,8
...,...,...,...,...,...,...,...,...,...,...
756,2022-03-18,-8,-14,-15,-2,-3,1,-2,3,-5
757,2022-03-19,-8,-14,-14,-2,-2,2,-2,4,-4
758,2022-03-20,-8,-14,-15,-2,-3,2,-2,4,-4
759,2022-03-21,-9,-14,-16,-2,-1,2,-1,4,-4


In [8]:
rrm_new_df.loc[rrm_new_df.NT.isna()]

,Date,AUS,NSW,VIC,QLD,SA,WA,TAS,NT,ACT
178,2020-08-17,-22,-18,-56,-5,-7,-1,-19,None,-11
179,2020-08-18,-22,-18,-56,-5,-7,-1,-19,None,-11
180,2020-08-19,-22,-18,-56,-5,-7,-1,-19,None,-12
181,2020-08-20,-22,-18,-56,-6,-7,-1,-19,None,-12
182,2020-08-21,-22,-17,-56,-6,-8,-1,-19,None,-12
183,2020-08-22,-22,-17,-57,-6,-8,-1,-19,None,-13
184,2020-08-23,-22,-17,-57,-6,-8,-1,-18,None,-13
185,2020-08-24,-22,-17,-57,-7,-8,0,-18,None,-13
186,2020-08-25,-22,-17,-57,-7,-7,0,-18,None,-13
187,2020-08-26,-22,-17,-56,-7,-7,0,-18,None,-13


In [9]:
rrm_df = rrm.toDF('Date', 'AUS', 'NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT')
rrm_df = pd.DataFrame(rrm_df.collect(), columns=['Date', 'AUS', 'NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT'])
print( rrm_df.Date.iloc[190:195] )

190      29/08/20
191      30/08/20
192      31/08/20
193    2020-09-01
194      02/09/20
Name: Date, dtype: object


In [10]:
rrm_new_df.Date.iloc[193] = pd.to_datetime('2020-09-01').date()

In [11]:
pd.to_datetime('2020-09-01').date()

datetime.date(2020, 9, 1)

### 4. Countinue with the NT value

In [12]:
rrm_new_df.loc[rrm_new_df.NT.isna()]

,Date,AUS,NSW,VIC,QLD,SA,WA,TAS,NT,ACT
178,2020-08-17,-22,-18,-56,-5,-7,-1,-19,None,-11
179,2020-08-18,-22,-18,-56,-5,-7,-1,-19,None,-11
180,2020-08-19,-22,-18,-56,-5,-7,-1,-19,None,-12
181,2020-08-20,-22,-18,-56,-6,-7,-1,-19,None,-12
182,2020-08-21,-22,-17,-56,-6,-8,-1,-19,None,-12
183,2020-08-22,-22,-17,-57,-6,-8,-1,-19,None,-13
184,2020-08-23,-22,-17,-57,-6,-8,-1,-18,None,-13
185,2020-08-24,-22,-17,-57,-7,-8,0,-18,None,-13
186,2020-08-25,-22,-17,-57,-7,-7,0,-18,None,-13
187,2020-08-26,-22,-17,-56,-7,-7,0,-18,None,-13


In [13]:
rrm_new_df = rrm_new_df.fillna(0)
rrm_new_df.loc[rrm_new_df.NT.isna()]

,Date,AUS,NSW,VIC,QLD,SA,WA,TAS,NT,ACT


### Save to file

In [14]:
for area in ['AUS', 'NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT']:
    rrm_new_df[area] = rrm_new_df[area].astype('int')
rrm_new_df['Date'] = rrm_new_df['Date'].astype('datetime64[ns]')

In [15]:
rrm_new_df.dtypes

Date    datetime64[ns]
AUS              int64
NSW              int64
VIC              int64
QLD              int64
SA               int64
WA               int64
TAS              int64
NT               int64
ACT              int64
dtype: object

In [19]:
rrm_new_df.to_parquet('../data/curated/external_rrm.parquet')